In [1]:
import meb
from meb import utils
from meb import datasets
from meb import core
from meb import models

from functools import partial
from typing import List, Tuple

import numpy as np
import pandas as pd
from numba import jit, njit
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import timm
from tqdm import tqdm



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [8]:
class Config(core.Config):
    device = torch.device("cuda:0")
    num_workers = 0
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]

In [9]:
c = datasets.CrossDataset(optical_flow=True, resize=28)
df = c.data_frame
data = c.data

In [10]:
#Off-ApexNet
class MConfig(Config):
    epochs = 3000
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(models.OffApexNet, num_classes=len(Config.action_units))
    
#Only use u and v optical flows, skip the strain
core.CrossDatasetValidation(MConfig).validate_n_times(df, data[:, :2], n_times=5)

100%|█████████████████████████████████████████| 5/5 [9:00:17<00:00, 6483.56s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
85.4 & 83.0 & 89.5 & 54.5 & 50.7 & 69.0 & 66.9 & 58.2 & 63.6 & 65.4 & 58.6 & 67.9 & 67.7

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
62.7 & 69.7 & 69.9 & 62.9 & 66.2 & 66.5 & 66.3
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
74.9 & 70.2 & 86.3 & 13.5 & 3.3 & 44.5 & 36.6 & 18.3 & 32.0 & 37.7 & 19.0 & 38.2 & 39.5

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
30.5 & 43.1 & 43.6 & 30.6 & 37.1 & 36.8 & 37.0


In [8]:
#STSTNet
class MConfig(Config):
    epochs = 500
    model = partial(models.STSTNet, num_classes=len(Config.action_units))
    optimizer = partial(optim.Adam, lr=5e-5, weight_decay=1e-3)
    
core.CrossDatasetValidation(MConfig).validate_n_times(df, data, n_times=5)

100%|█████████████████████████████████████████| 5/5 [2:09:59<00:00, 1559.96s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
85.0 & 82.2 & 89.2 & 49.8 & 52.4 & 60.3 & 57.8 & 53.5 & 55.6 & 60.9 & 55.1 & 62.9 & 63.7

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
62.5 & 67.3 & 65.3 & 59.0 & 62.6 & 62.6 & 63.2
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
74.2 & 68.8 & 85.7 & 3.6 & 6.4 & 29.0 & 18.6 & 9.0 & 16.7 & 29.0 & 11.8 & 28.3 & 31.8

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
29.9 & 38.3 & 34.9 & 23.1 & 30.2 & 29.2 & 31.0


In [11]:
c = datasets.CrossDataset(optical_flow=True, resize=60)
df = c.data_frame
data = c.data

In [13]:
#RCN-A
class MConfig(Config):
    epochs = 500
    model = partial(models.RCN_A, num_input=3, featuremaps=64, num_classes=len(Config.action_units))
    optimizer = partial(optim.SGD, lr=1e-4, weight_decay=5e-4, momentum=0.9)
    
core.CrossDatasetValidation(MConfig).validate_n_times(df, data, n_times=5)

100%|█████████████████████████████████████████| 5/5 [5:19:38<00:00, 3835.69s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
85.2 & 83.1 & 88.6 & 49.8 & 49.3 & 55.5 & 56.3 & 49.1 & 57.8 & 58.0 & 49.4 & 60.7 & 61.9

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
59.6 & 65.3 & 64.3 & 59.7 & 60.7 & 61.0 & 61.8
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
74.8 & 71.0 & 85.3 & 4.7 & 0.0 & 24.1 & 15.9 & 0.0 & 21.1 & 23.7 & 0.0 & 24.0 & 28.7

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
24.2 & 34.6 & 33.5 & 24.7 & 26.4 & 27.1 & 28.4


In [15]:
#SSSNet
c = datasets.CrossDataset(optical_flow=True, resize=64)
df = c.data_frame
data = c.data

class MConfig(Config):
    epochs = 200
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(models.SSSNet, num_classes=len(Config.action_units))
    
core.CrossDatasetValidation(MConfig).validate_n_times(df, data, n_times=5)

100%|██████████████████████████████████████████| 5/5 [1:06:55<00:00, 803.18s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
85.2 & 84.0 & 90.5 & 54.8 & 51.9 & 71.1 & 58.6 & 58.6 & 65.8 & 66.9 & 60.6 & 71.3 & 68.3

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
64.2 & 71.4 & 69.2 & 64.7 & 66.5 & 66.4 & 67.1
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
74.6 & 72.1 & 87.5 & 13.6 & 5.3 & 48.6 & 20.3 & 19.3 & 36.2 & 40.8 & 22.7 & 44.7 & 40.5

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
33.1 & 46.3 & 42.3 & 34.3 & 38.1 & 36.2 & 38.4


In [2]:
c = datasets.CrossDataset(optical_flow=True, resize=112)
df = c.data_frame
data = c.data

In [9]:
class MConfig(Config):
    epochs = 50
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet34, num_classes=len(Config.action_units))

In [11]:
core.CrossDatasetValidation(MConfig).validate_n_times(df, data, n_times=5)

100%|█████████████████████████████████████████| 5/5 [2:02:36<00:00, 1471.38s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
82.6 & 76.5 & 86.4 & 50.9 & 50.0 & 60.6 & 52.2 & 54.5 & 56.9 & 61.4 & 54.1 & 63.4 & 62.5

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
58.4 & 67.2 & 61.4 & 60.0 & 62.0 & 60.6 & 61.6
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
69.9 & 58.3 & 82.1 & 6.2 & 1.7 & 30.1 & 7.9 & 11.4 & 19.8 & 30.6 & 9.7 & 29.1 & 29.7

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
22.2 & 38.4 & 27.8 & 25.5 & 29.8 & 25.6 & 28.2


In [3]:
class Config(core.Config):
    epochs = 50
    device = torch.device("cuda:1")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    num_workers = 0
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]
    model = partial(timm.models.resnet34, num_classes=len(core.Config.action_units), pretrained=True)

In [5]:
core.CrossDatasetValidator(Config).validate_n_times(df, data, n_times=5)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9974 | test_mean: 0.5905
Test per AU: [('AU1', 74.53), ('AU2', 75.07), ('AU4', 80.43), ('AU5', 49.19), ('AU6', 49.06), ('AU7', 53.8), ('AU9', 58.97), ('AU10', 49.47), ('AU12', 57.99), ('AU14', 50.74), ('AU15', 53.8), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9952 | test_mean: 0.2348
Test per AU: [('AU1', 54.05), ('AU2', 55.81), ('AU4', 74.02), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 11.76), ('AU9', 28.57), ('AU10', 0.0), ('AU12', 23.53), ('AU14', 8.0), ('AU15', 11.76), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9959 | test_mean: 0.7058
Test per AU: [('AU1', 90.56), ('AU2', 85.02), ('AU4', 93.36), ('AU5', 49.8), ('AU6', 61.29), ('AU7', 71.94), ('AU9', 54.84), ('AU10', 59.69), ('AU12', 76.57), ('AU14', 76.45), ('AU15', 48.39), ('AU17', 79.08)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9921 | test_mean: 0.4477
Test per AU: [('AU1', 83.33), ('AU2', 73.08), ('A

 20%|████████▏                                | 1/5 [31:42<2:06:51, 1902.86s/it]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9934 | test_mean: 0.5986
Test per AU: [('AU1', 70.09), ('AU2', 76.86), ('AU4', 77.56), ('AU5', 49.19), ('AU6', 49.33), ('AU7', 43.81), ('AU9', 44.08), ('AU10', 66.13), ('AU12', 66.1), ('AU14', 58.5), ('AU15', 54.88), ('AU17', 61.82)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9882 | test_mean: 0.2634
Test per AU: [('AU1', 45.71), ('AU2', 57.14), ('AU4', 72.22), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 6.35), ('AU9', 0.0), ('AU10', 33.33), ('AU12', 37.04), ('AU14', 24.24), ('AU15', 13.33), ('AU17', 26.67)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9965 | test_mean: 0.6749
Test per AU: [('AU1', 90.56), ('AU2', 87.16), ('AU4', 95.31), ('AU5', 49.61), ('AU6', 48.59), ('AU7', 53.31), ('AU9', 48.59), ('AU10', 56.75), ('AU12', 75.8), ('AU14', 73.69), ('AU15', 54.37), ('AU17', 76.1)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9938 | test_mean: 0.3866
Test per AU: [('AU1', 83.33), ('AU2', 76.92), ('A

 40%|███████████████▌                       | 2/5 [1:08:28<1:44:03, 2081.18s/it]

...

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9982 | test_mean: 0.6373
Test per AU: [('AU1', 68.25), ('AU2', 77.75), ('AU4', 85.07), ('AU5', 47.93), ('AU6', 49.87), ('AU7', 53.8), ('AU9', 62.6), ('AU10', 66.13), ('AU12', 69.91), ('AU14', 55.62), ('AU15', 72.27), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9965 | test_mean: 0.3209
Test per AU: [('AU1', 41.38), ('AU2', 58.33), ('AU4', 80.88), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 11.76), ('AU9', 35.29), ('AU10', 33.33), ('AU12', 43.48), ('AU14', 18.75), ('AU15', 47.62), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9986 | test_mean: 0.6926
Test per AU: [('AU1', 93.3), ('AU2', 84.22), ('AU4', 95.7), ('AU5', 49.51), ('AU6', 48.59), ('AU7', 71.68), ('AU9', 48.7), ('AU10', 53.94), ('AU12', 76.05), ('AU14', 72.78), ('AU15', 59.69), ('AU17', 77.0)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9975 | test_mean: 0.4205
Test per AU: [('AU1', 88.14), ('AU2', 71.7), ('A

 80%|████████████████████████████████▊        | 4/5 [2:23:37<36:42, 2202.77s/it]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9993 | test_mean: 0.6159
Test per AU: [('AU1', 76.38), ('AU2', 80.67), ('AU4', 81.54), ('AU5', 49.47), ('AU6', 54.32), ('AU7', 51.06), ('AU9', 49.11), ('AU10', 49.33), ('AU12', 64.41), ('AU14', 60.17), ('AU15', 60.84), ('AU17', 61.82)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9987 | test_mean: 0.2841
Test per AU: [('AU1', 57.14), ('AU2', 64.52), ('AU4', 77.24), ('AU5', 0.0), ('AU6', 12.5), ('AU7', 8.33), ('AU9', 9.52), ('AU10', 0.0), ('AU12', 33.33), ('AU14', 26.67), ('AU15', 25.0), ('AU17', 26.67)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9966 | test_mean: 0.6757
Test per AU: [('AU1', 92.45), ('AU2', 84.22), ('AU4', 93.75), ('AU5', 49.71), ('AU6', 68.78), ('AU7', 57.01), ('AU9', 48.49), ('AU10', 53.94), ('AU12', 70.11), ('AU14', 69.32), ('AU15', 48.28), ('AU17', 74.8)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9937 | test_mean: 0.3931
Test per AU: [('AU1', 86.67), ('AU2', 71.7), ('

100%|█████████████████████████████████████████| 5/5 [3:01:59<00:00, 2183.93s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
83.9 & 83.7 & 89.2 & 53.5 & 51.7 & 65.9 & 56.3 & 55.8 & 65.8 & 66.3 & 56.6 & 68.3 & 66.4

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
61.5 & 68.7 & 65.9 & 64.4 & 64.8 & 64.8 & 65.0
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
72.1 & 71.7 & 85.8 & 11.2 & 5.4 & 38.7 & 16.0 & 13.5 & 36.5 & 39.9 & 14.8 & 39.2 & 37.1

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
28.2 & 41.2 & 36.3 & 34.0 & 35.0 & 33.2 & 34.7


In [27]:
core.CrossDatasetValidator(MConfig).validate_n_times(df, data, n_times=5)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.998 | test_mean: 0.5919
Test per AU: [('AU1', 79.53), ('AU2', 80.73), ('AU4', 75.58), ('AU5', 49.19), ('AU6', 49.87), ('AU7', 55.49), ('AU9', 46.65), ('AU10', 49.6), ('AU12', 59.96), ('AU14', 53.71), ('AU15', 48.08), ('AU17', 61.82)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9963 | test_mean: 0.231
Test per AU: [('AU1', 62.86), ('AU2', 64.0), ('AU4', 66.67), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 14.29), ('AU9', 4.88), ('AU10', 0.0), ('AU12', 23.53), ('AU14', 14.29), ('AU15', 0.0), ('AU17', 26.67)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9962 | test_mean: 0.6716
Test per AU: [('AU1', 93.3), ('AU2', 81.95), ('AU4', 92.95), ('AU5', 49.71), ('AU6', 48.7), ('AU7', 76.67), ('AU9', 48.7), ('AU10', 48.28), ('AU12', 67.11), ('AU14', 76.85), ('AU15', 54.37), ('AU17', 67.38)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.993 | test_mean: 0.3796
Test per AU: [('AU1', 88.14), ('AU2', 67.86), ('AU4', 9

 20%|████████▊                                   | 1/5 [14:51<59:24, 891.01s/it]

[[59.18514834284382, 67.16368685580647, 60.399706100661824, 60.841168662079056, 61.77620165870836, 61.444570410658905, 61.80174700512641], [23.09744710892487, 37.95737813598432, 25.06993813207108, 26.65907695569131, 28.60712571926162, 27.02486252056802, 28.069304762083544]]
MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9983 | test_mean: 0.5788
Test per AU: [('AU1', 77.38), ('AU2', 73.31), ('AU4', 77.0), ('AU5', 49.73), ('AU6', 49.73), ('AU7', 58.91), ('AU9', 46.65), ('AU10', 49.06), ('AU12', 55.51), ('AU14', 53.71), ('AU15', 48.08), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9968 | test_mean: 0.2065
Test per AU: [('AU1', 58.82), ('AU2', 50.0), ('AU4', 68.85), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 20.0), ('AU9', 4.88), ('AU10', 0.0), ('AU12', 16.67), ('AU14', 14.29), ('AU15', 0.0), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9913 | test_mean: 0.6317
Test per AU: [('AU1', 89.97), ('AU2', 78.15), ('AU4', 95.31), ('AU5',

 40%|█████████████████▏                         | 2/5 [35:17<54:24, 1088.21s/it]

...

[[58.66493208008902, 64.0686968694479, 59.82186836848994, 60.17187481849488, 61.669312880834525, 61.37400005729494, 60.9617808457752], [22.175296714603896, 31.926492749871667, 24.01084468460355, 25.840963648357434, 28.603565727195296, 26.70266144502202, 26.543304161608976]]
MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9935 | test_mean: 0.5687
Test per AU: [('AU1', 81.58), ('AU2', 63.04), ('AU4', 77.53), ('AU5', 49.73), ('AU6', 49.87), ('AU7', 67.23), ('AU9', 44.08), ('AU10', 49.47), ('AU12', 56.97), ('AU14', 46.61), ('AU15', 48.08), ('AU17', 48.22)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9879 | test_mean: 0.184
Test per AU: [('AU1', 66.67), ('AU2', 30.0), ('AU4', 71.11), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 36.36), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 16.67), ('AU14', 0.0), ('AU15', 0.0), ('AU17', 0.0)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9966 | test_mean: 0.6515
Test per AU: [('AU1', 90.28), ('AU2', 83.67), ('AU4', 93.75), ('AU5', 49.

 80%|████████████████████████████████▊        | 4/5 [1:19:02<20:21, 1221.96s/it]

[[56.86713543444786, 65.14800321410532, 60.67183194555237, 59.37924129252261, 60.37624420615765, 59.43248179550954, 60.31248964804923], [18.400673400673405, 34.287560059839365, 25.80062627419426, 24.155852489940596, 27.022431539691667, 22.906227867216124, 25.428895271925906]]
MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9953 | test_mean: 0.5654
Test per AU: [('AU1', 72.69), ('AU2', 71.04), ('AU4', 76.14), ('AU5', 49.6), ('AU6', 49.87), ('AU7', 55.49), ('AU9', 44.08), ('AU10', 49.06), ('AU12', 56.19), ('AU14', 50.74), ('AU15', 48.08), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9913 | test_mean: 0.1784
Test per AU: [('AU1', 50.0), ('AU2', 45.45), ('AU4', 66.67), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 14.29), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 15.38), ('AU14', 8.0), ('AU15', 0.0), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9931 | test_mean: 0.6464
Test per AU: [('AU1', 87.12), ('AU2', 81.37), ('AU4', 91.79), ('AU5',

100%|█████████████████████████████████████████| 5/5 [1:39:39<00:00, 1195.88s/it]

[[56.53887703894694, 64.64159393608332, 59.077493386621484, 59.25270284796045, 63.144581848511, 61.885946172848136, 60.75686587182856], [17.83977133977134, 33.6405720182493, 23.245180319648405, 23.844201886343193, 31.724999052599266, 27.841892225011172, 26.356102806937113]]
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
82.2 & 78.6 & 87.1 & 49.2 & 49.5 & 59.6 & 51.1 & 51.7 & 58.0 & 60.5 & 51.9 & 59.5 & 61.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
57.8 & 64.8 & 60.2 & 60.0 & 62.0 & 60.8 & 60.9
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
68.9 & 62.3 & 83.0 & 2.1 & 0.6 & 27.9 & 5.4 & 5.3 & 21.3 & 28.3 & 5.2 & 21.3 & 27.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
20.4 & 33.6 & 24.9 & 25.4 & 29.5 & 25.7 & 26.6


In [9]:
class MConfig(Config):
    epochs = 50
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet18, num_classes=len(Config.action_units))

In [22]:
class Config(core.Config):
    epochs = 50
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    num_workers = 0
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]
    model = partial(timm.models.resnet18, num_classes=len(core.Config.action_units), pretrained=True)

In [24]:
core.CrossDatasetValidator(Config).validate_n_times(df, data, n_times=5)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9997 | test_mean: 0.5936
Test per AU: [('AU1', 79.53), ('AU2', 77.75), ('AU4', 79.07), ('AU5', 49.33), ('AU6', 49.6), ('AU7', 51.74), ('AU9', 46.65), ('AU10', 49.6), ('AU12', 65.0), ('AU14', 53.71), ('AU15', 54.88), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9995 | test_mean: 0.2368
Test per AU: [('AU1', 62.86), ('AU2', 58.33), ('AU4', 73.76), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 9.09), ('AU9', 4.88), ('AU10', 0.0), ('AU12', 33.33), ('AU14', 14.29), ('AU15', 13.33), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9992 | test_mean: 0.6857
Test per AU: [('AU1', 91.63), ('AU2', 86.32), ('AU4', 94.14), ('AU5', 49.71), ('AU6', 48.7), ('AU7', 67.89), ('AU9', 48.59), ('AU10', 54.37), ('AU12', 80.01), ('AU14', 74.05), ('AU15', 48.39), ('AU17', 79.08)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9984 | test_mean: 0.4059
Test per AU: [('AU1', 85.25), ('AU2', 75.47), ('AU

 20%|████████▍                                 | 1/5 [16:25<1:05:42, 985.70s/it]

[[59.36375111992592, 68.57191839552829, 67.56036024784234, 63.6769849483867, 65.56845680794623, 65.40330262238251, 65.02412902366866], [23.679699545682944, 40.590765046182355, 38.85506947329702, 32.21952867245769, 35.9411931312488, 34.47262144077816, 34.2931462182745]]
MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.5848
Test per AU: [('AU1', 77.38), ('AU2', 78.42), ('AU4', 81.62), ('AU5', 49.19), ('AU6', 49.73), ('AU7', 56.02), ('AU9', 44.08), ('AU10', 49.47), ('AU12', 59.17), ('AU14', 53.71), ('AU15', 48.08), ('AU17', 54.88)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.2193
Test per AU: [('AU1', 58.82), ('AU2', 60.61), ('AU4', 76.47), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 17.39), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 22.22), ('AU14', 14.29), ('AU15', 0.0), ('AU17', 13.33)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9939 | test_mean: 0.6831
Test per AU: [('AU1', 90.82), ('AU2', 82.5), ('AU4', 96.48), ('AU5', 49.61), (

 40%|█████████████████▌                          | 2/5 [31:24<46:44, 934.86s/it]

...

[[59.38958700813929, 70.20457377129405, 68.47407354462362, 64.93128792378998, 65.76974509608804, 65.78117963436934, 65.75840782971738], [23.78997545664213, 43.86944673435202, 40.76292774555357, 34.75898281922101, 36.52215712700544, 34.87478511919213, 35.763045833661046]]
MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.6151
Test per AU: [('AU1', 83.52), ('AU2', 79.35), ('AU4', 80.21), ('AU5', 49.87), ('AU6', 49.6), ('AU7', 58.44), ('AU9', 51.45), ('AU10', 49.6), ('AU12', 66.22), ('AU14', 54.13), ('AU15', 54.88), ('AU17', 60.84)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.2755
Test per AU: [('AU1', 70.27), ('AU2', 61.54), ('AU4', 74.24), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 21.05), ('AU9', 13.95), ('AU10', 0.0), ('AU12', 36.36), ('AU14', 14.81), ('AU15', 13.33), ('AU17', 25.0)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9999 | test_mean: 0.6916
Test per AU: [('AU1', 92.22), ('AU2', 87.16), ('AU4', 94.14), ('AU5', 49.71

 80%|█████████████████████████████████▌        | 4/5 [1:02:40<15:38, 938.69s/it]

[[61.50880960432127, 69.16344490471538, 72.06504132268208, 61.33075934163696, 63.40294996925556, 64.29528142986656, 65.29438109541297], [27.547421709498416, 41.98307513917552, 47.96433523252513, 28.492553781346153, 31.986296896141454, 32.05502406260354, 35.004784470215036]]
MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9993 | test_mean: 0.6175
Test per AU: [('AU1', 77.38), ('AU2', 80.73), ('AU4', 80.35), ('AU5', 49.87), ('AU6', 49.6), ('AU7', 55.49), ('AU9', 49.11), ('AU10', 49.6), ('AU12', 66.12), ('AU14', 54.57), ('AU15', 60.84), ('AU17', 67.37)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9986 | test_mean: 0.2787
Test per AU: [('AU1', 58.82), ('AU2', 64.0), ('AU4', 74.63), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 14.29), ('AU9', 9.52), ('AU10', 0.0), ('AU12', 35.29), ('AU14', 15.38), ('AU15', 25.0), ('AU17', 37.5)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9996 | test_mean: 0.6966
Test per AU: [('AU1', 95.86), ('AU2', 88.02), ('AU4', 96.09), ('AU5',

100%|██████████████████████████████████████████| 5/5 [1:17:48<00:00, 933.76s/it]

[[61.75162352532318, 69.66142576472714, 67.8686125018704, 62.929868246307585, 65.68807336945838, 66.1421644332552, 65.67362797349033], [27.869887660383707, 42.66951873757214, 39.32320019252197, 31.25708642043431, 36.38686630299511, 35.466881872534756, 35.49557353107367]]
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
86.2 & 84.1 & 90.2 & 54.4 & 51.9 & 65.6 & 52.6 & 58.3 & 66.4 & 63.2 & 55.5 & 71.3 & 66.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
60.1 & 69.2 & 69.3 & 63.9 & 65.2 & 65.2 & 65.5
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
76.2 & 72.1 & 87.1 & 12.6 & 5.5 & 38.2 & 8.6 & 18.4 & 37.3 & 33.7 & 12.4 & 44.7 & 37.2

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
25.0 & 41.9 & 42.4 & 33.1 & 35.4 & 33.7 & 35.2


In [12]:
core.CrossDatasetValidator(MConfig).validate_n_times(df, data, n_times=5)

 20%|████████▊                                   | 1/5 [13:17<53:08, 797.19s/it]

[[59.18514834284382, 67.16368685580647, 60.399706100661824, 60.841168662079056, 61.77620165870836, 61.444570410658905, 61.80174700512641], [23.09744710892487, 37.95737813598432, 25.06993813207108, 26.65907695569131, 28.60712571926162, 27.02486252056802, 28.069304762083544]]


 40%|█████████████████▌                          | 2/5 [25:56<38:44, 774.94s/it]

[[57.88090964175016, 63.165157637126946, 60.871767708659775, 60.54262397002562, 63.0746317673619, 59.952537616394594, 60.91460472355317], [20.649344370561764, 30.32291406361251, 26.320506455546848, 26.41620452894962, 31.593986311815915, 24.2392025426471, 26.59035971218896]]


 60%|██████████████████████████▍                 | 3/5 [38:31<25:31, 765.89s/it]

[[58.66493208008902, 64.0686968694479, 59.82186836848994, 60.17187481849488, 61.669312880834525, 61.37400005729494, 60.9617808457752], [22.175296714603896, 31.926492749871667, 24.01084468460355, 25.840963648357434, 28.603565727195296, 26.70266144502202, 26.543304161608976]]


 80%|███████████████████████████████████▏        | 4/5 [51:13<12:44, 764.11s/it]

[[56.86713543444786, 65.14800321410532, 60.67183194555237, 59.37924129252261, 60.37624420615765, 59.43248179550954, 60.31248964804923], [18.400673400673405, 34.287560059839365, 25.80062627419426, 24.155852489940596, 27.022431539691667, 22.906227867216124, 25.428895271925906]]


 80%|███████████████████████████████████▏        | 4/5 [53:07<13:16, 796.75s/it]


KeyboardInterrupt: 

In [11]:
core.CrossDatasetValidator(MConfig).validate_n_times(df, data, n_times=5)

100%|██████████████████████████████████████████| 5/5 [1:14:55<00:00, 899.20s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
82.2 & 78.6 & 87.1 & 49.2 & 49.5 & 59.6 & 51.1 & 51.7 & 58.0 & 60.5 & 51.9 & 59.5 & 61.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
57.8 & 64.8 & 60.2 & 60.0 & 62.0 & 60.8 & 60.9
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
68.9 & 62.3 & 83.0 & 2.1 & 0.6 & 27.9 & 5.4 & 5.3 & 21.3 & 28.3 & 5.2 & 21.3 & 27.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
20.4 & 33.6 & 24.9 & 25.4 & 29.5 & 25.7 & 26.6


In [13]:
core.CrossDatasetValidation(MConfig).validate_n_times(df, data, n_times=5)

100%|█████████████████████████████████████████| 5/5 [2:08:29<00:00, 1541.80s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
82.2 & 78.6 & 87.1 & 49.2 & 49.5 & 59.6 & 51.1 & 51.7 & 58.0 & 60.5 & 51.9 & 59.5 & 61.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
57.8 & 64.8 & 60.2 & 60.0 & 62.0 & 60.8 & 60.9
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
68.9 & 62.3 & 83.0 & 2.1 & 0.6 & 27.9 & 5.4 & 5.3 & 21.3 & 28.3 & 5.2 & 21.3 & 27.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
20.4 & 33.6 & 24.9 & 25.4 & 29.5 & 25.7 & 26.6


In [28]:
class Config(core.Config):
    epochs = 50
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    num_workers = 0
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]
    model = partial(timm.models.resnet10t, num_classes=len(core.Config.action_units), pretrained=True)

In [29]:
core.CrossDatasetValidator(Config).validate_n_times(df, data, n_times=5)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9931 | test_mean: 0.576
Test per AU: [('AU1', 73.24), ('AU2', 84.57), ('AU4', 76.05), ('AU5', 49.6), ('AU6', 49.87), ('AU7', 47.65), ('AU9', 46.43), ('AU10', 49.6), ('AU12', 54.88), ('AU14', 55.62), ('AU15', 55.53), ('AU17', 48.22)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9866 | test_mean: 0.2145
Test per AU: [('AU1', 51.43), ('AU2', 71.43), ('AU4', 68.7), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 0.0), ('AU9', 4.76), ('AU10', 0.0), ('AU12', 13.33), ('AU14', 18.75), ('AU15', 28.95), ('AU17', 0.0)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9996 | test_mean: 0.6898
Test per AU: [('AU1', 91.63), ('AU2', 83.44), ('AU4', 94.92), ('AU5', 49.51), ('AU6', 48.59), ('AU7', 75.32), ('AU9', 48.59), ('AU10', 54.37), ('AU12', 66.28), ('AU14', 74.31), ('AU15', 54.37), ('AU17', 86.44)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9992 | test_mean: 0.415
Test per AU: [('AU1', 85.25), ('AU2', 70.37), ('AU4', 

 20%|████████▏                                | 1/5 [18:18<1:13:13, 1098.30s/it]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.5851
Test per AU: [('AU1', 80.53), ('AU2', 79.35), ('AU4', 77.31), ('AU5', 49.47), ('AU6', 49.6), ('AU7', 53.8), ('AU9', 43.92), ('AU10', 49.47), ('AU12', 62.91), ('AU14', 52.2), ('AU15', 48.08), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.2187
Test per AU: [('AU1', 64.86), ('AU2', 61.54), ('AU4', 68.91), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 11.76), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 28.57), ('AU14', 12.5), ('AU15', 0.0), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 1.0 | test_mean: 0.6773
Test per AU: [('AU1', 89.76), ('AU2', 81.95), ('AU4', 94.53), ('AU5', 49.71), ('AU6', 62.23), ('AU7', 70.83), ('AU9', 48.7), ('AU10', 54.37), ('AU12', 72.41), ('AU14', 72.5), ('AU15', 48.39), ('AU17', 67.38)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 1.0 | test_mean: 0.3921
Test per AU: [('AU1', 81.97), ('AU2', 67.86), ('AU4', 94.35), ('AU

 40%|█████████████████▏                         | 2/5 [36:44<55:08, 1102.69s/it]

...

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.5756
Test per AU: [('AU1', 77.38), ('AU2', 84.57), ('AU4', 72.97), ('AU5', 49.19), ('AU6', 49.87), ('AU7', 52.9), ('AU9', 43.92), ('AU10', 49.6), ('AU12', 56.56), ('AU14', 50.14), ('AU15', 48.08), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.2013
Test per AU: [('AU1', 58.82), ('AU2', 71.43), ('AU4', 60.95), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 10.53), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 18.18), ('AU14', 7.41), ('AU15', 0.0), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 1.0 | test_mean: 0.6726
Test per AU: [('AU1', 92.22), ('AU2', 84.22), ('AU4', 94.53), ('AU5', 49.61), ('AU6', 55.94), ('AU7', 75.32), ('AU9', 48.7), ('AU10', 53.94), ('AU12', 64.21), ('AU14', 75.46), ('AU15', 48.39), ('AU17', 64.59)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 1.0 | test_mean: 0.3817
Test per AU: [('AU1', 86.21), ('AU2', 71.7), ('AU4', 94.49), ('A

 80%|████████████████████████████████▊        | 4/5 [1:13:28<18:22, 1102.19s/it]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.5858
Test per AU: [('AU1', 71.78), ('AU2', 82.05), ('AU4', 76.55), ('AU5', 49.87), ('AU6', 49.33), ('AU7', 57.86), ('AU9', 46.65), ('AU10', 49.47), ('AU12', 69.52), ('AU14', 46.31), ('AU15', 48.08), ('AU17', 55.49)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 1.0 | test_mean: 0.2199
Test per AU: [('AU1', 48.48), ('AU2', 66.67), ('AU4', 69.23), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 18.18), ('AU9', 4.88), ('AU10', 0.0), ('AU12', 42.11), ('AU14', 0.0), ('AU15', 0.0), ('AU17', 14.29)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9982 | test_mean: 0.6915
Test per AU: [('AU1', 92.87), ('AU2', 85.84), ('AU4', 92.57), ('AU5', 49.8), ('AU6', 66.54), ('AU7', 75.18), ('AU9', 48.7), ('AU10', 54.37), ('AU12', 71.59), ('AU14', 73.34), ('AU15', 54.37), ('AU17', 64.59)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9969 | test_mean: 0.4197
Test per AU: [('AU1', 87.27), ('AU2', 74.51), ('AU4', 92.

100%|█████████████████████████████████████████| 5/5 [1:31:52<00:00, 1102.48s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
82.0 & 80.3 & 87.8 & 52.3 & 52.3 & 66.2 & 53.7 & 52.4 & 63.0 & 61.2 & 53.5 & 68.7 & 64.4

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
58.0 & 68.1 & 66.3 & 64.3 & 63.5 & 61.9 & 63.7
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
68.7 & 65.2 & 83.8 & 8.3 & 6.4 & 39.6 & 11.0 & 6.8 & 31.0 & 29.6 & 8.8 & 39.6 & 33.2

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
21.2 & 39.8 & 36.7 & 33.8 & 32.6 & 27.7 & 32.0
